# Ensembles

## Ensemble reduction

This tutorial will explore ensemble reduction (also known as ensemble selection) using `xscen`. This will use pre-computed annual mean temperatures from `xclim.testing`.

In [ ]:
from xclim.testing import open_dataset

import xscen as xs

datasets = {
    "ACCESS": "EnsembleStats/BCCAQv2+ANUSPLIN300_ACCESS1-0_historical+rcp45_r1i1p1_1950-2100_tg_mean_YS.nc",
    "BNU-ESM": "EnsembleStats/BCCAQv2+ANUSPLIN300_BNU-ESM_historical+rcp45_r1i1p1_1950-2100_tg_mean_YS.nc",
    "CCSM4-r1": "EnsembleStats/BCCAQv2+ANUSPLIN300_CCSM4_historical+rcp45_r1i1p1_1950-2100_tg_mean_YS.nc",
    "CCSM4-r2": "EnsembleStats/BCCAQv2+ANUSPLIN300_CCSM4_historical+rcp45_r2i1p1_1950-2100_tg_mean_YS.nc",
    "CNRM-CM5": "EnsembleStats/BCCAQv2+ANUSPLIN300_CNRM-CM5_historical+rcp45_r1i1p1_1970-2050_tg_mean_YS.nc",
}

for d in datasets:
    ds = open_dataset(datasets[d]).isel(lon=slice(0, 4), lat=slice(0, 4))
    ds = xs.climatological_mean(ds, window=30, periods=[[1981, 2010], [2021, 2050]])
    datasets[d] = xs.compute_deltas(ds, reference_horizon="1981-2010")
    datasets[d].attrs["cat:id"] = d  # Required by build_reduction_data
    datasets[d].attrs["cat:xrfreq"] = "AS-JAN"

### Preparing the data

Ensemble reduction is built upon climate indicators that are relevant to represent the ensemble's variability for a given application. In this case, we'll use the mean temperature delta between 2021-2050 and 1981-2010.

However, the functions implemented in `xclim.ensembles._reduce` require a very specific 2-D DataArray of dimensions "realization" and "criteria". That means that all the variables need to be combined and renamed, and that all dimensions need to be stacked together.

`xs.build_reduction_data` can be used to prepare the data for ensemble reduction. Its arguments are:

- `datasets` (dict, list)
- `xrfreqs` are the unique frequencies of the indicators.
- `horizons` is used to instruct on which horizon(s) to build the data from.

Because a simulation could have multiple datasets (in the case of multiple frequencies), an attempt will be made to decipher the ID and frequency from the metadata.

In [ ]:
data = xs.build_reduction_data(
    datasets=datasets,
    xrfreqs=["AS-JAN"],
    horizons=["2021-2050"],
)

data

The number of criteria corresponds to: `indicators x horizons x longitude x latitude`, but criteria that are purely NaN across all realizations are removed.

Note that `xs.spatial_mean` could have been used prior to calling that function to remove the spatial dimensions.

### Selecting a reduced ensemble

<div class="alert alert-info"> <b>NOTE</b>
    
Ensemble reduction in `xscen` is built upon `xclim.ensembles`. For more information on basic usage and available methods, [please consult their documentation](https://xclim.readthedocs.io/en/stable/notebooks/ensembles-advanced.html).
</div>

Ensemble reduction through `xscen.reduce_ensemble` consists in a simple call to `xclim`. The arguments are:
- `data`, which is the 2D DataArray that is created by using `xs.build_reduction_data`.
- `method` is either `kkz` or `kmeans`. See the link above for further details on each technique.
- `kwargs` is a dictionary of arguments to send to the method chosen.

In [ ]:
selected, clusters, fig_data = xs.reduce_ensemble(
    data=data, method="kmeans", kwargs={"method": {"n_clusters": 3}}
)

The method always returns 3 outputs (selected, clusters, fig_data):
- `selected` is a DataArray of dimension 'realization' listing the selected simulations.
- `clusters` (kmeans only) groups every realization in their respective clusters in a python dictionary.
- `fig_data` (kmeans only) can be used to call `xclim.ensembles.plot_rsqprofile(fig_data)`

In [ ]:
selected

In [ ]:
# To see the clusters in more details
clusters

In [ ]:
from xclim.ensembles import plot_rsqprofile

plot_rsqprofile(fig_data)

## Ensemble partition
This tutorial will show how to use xscen to create the input for [xclim partition functions](https://xclim.readthedocs.io/en/stable/api.html#uncertainty-partitioning).

In [ ]:
# Get catalog
from pathlib import Path

output_folder = Path().absolute() / "_data"
cat = xs.DataCatalog(str(output_folder / "tutorial-catalog.json"))

# create a dictionnary of datasets wanted for the partition
input_dict = cat.search(variable="tas", member="r1i1p1f1").to_dataset_dict(
    xarray_open_kwargs={"engine": "h5netcdf"}
)

From a dictionary of datasets, the function creates a dataset with new dimensions in `partition_dim`(`["source", "experiment", "bias_adjust_project"]`, if they exist). In this toy example, we only have different experiments.
- By default, it translates the xscen vocabulary (eg. `experiment`) to the xclim partition vocabulary (eg. `scenario`). It is possible to pass `rename_dict` to rename the dimensions with other names.
- If the inputs are not on the same grid, they can be regridded through `regrid_kw` or subset to a point through `subset_kw`. The functions assumes that if there are different `bias_adjust_project`, they will be on different grids (with all `source` on the same grid). If there is one or less `bias_adjust_project`, the assumption is that`source` have different grids.
- You can also compute indicators on the data if the input is daily. This can be especially useful when the daily input data is on different calendars.

In [ ]:
# build a single dataset
import xclim as xc

ds = xs.ensembles.build_partition_data(
    input_dict,
    subset_kw=dict(name="mtl", method="gridpoint", lat=[45.5], lon=[-73.6]),
    indicators_kw={"indicators": [xc.atmos.tg_mean]},
)
ds

Pass the input to an xclim partition function.

In [ ]:
# This is a hidden cell.
# extend with fake data to have at least 3 years
import xarray as xr

ds2 = ds.copy()
ds["time"] = xr.cftime_range(start="2001-01-01", periods=len(ds["time"]), freq="YS")
ds2["time"] = xr.cftime_range(start="2003-01-01", periods=len(ds["time"]), freq="YS")
ds = xr.concat([ds, ds2], dim="time")

In [ ]:
# compute uncertainty partitionning
mean, uncertainties = xc.ensembles.hawkins_sutton(ds.tg_mean)
uncertainties

<div class="alert alert-info"> <b>NOTE</b>
    
Note that the [figanos library](https://figanos.readthedocs.io/en/latest/) provides a function `fg.partition` to plot the uncertainties.
    
</div>